## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-07-19-52-14 +0000,nypost,Top White House economic advisor Kevin Hassett...,https://nypost.com/2025/09/07/business/white-h...
1,2025-09-07-19-43-09 +0000,wapo,Trump border czar warns sanctuary cities natio...,https://www.washingtonpost.com/nation/2025/09/...
2,2025-09-07-19-32-08 +0000,nypost,Hochul claims kids are better off without phon...,https://nypost.com/2025/09/07/us-news/hochul-c...
3,2025-09-07-19-30-00 +0000,missionlocal,Illustrating a day inside San Francisco’s immi...,https://missionlocal.org/2025/09/illustrating-...
4,2025-09-07-19-27-59 +0000,nyt,Trump Is Met With Mostly Boos at U.S. Open as ...,https://www.nytimes.com/2025/09/07/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2312/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
11,trump,22
134,ukraine,11
104,government,9
48,raid,9
102,russia,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
93,2025-09-07-12-19-07 +0000,nypost,South Korean workers arrested in ICE raid at G...,https://nypost.com/2025/09/07/us-news/south-ko...,83
71,2025-09-07-15-08-17 +0000,nypost,Russian attack on Ukraine’s government HQ in l...,https://nypost.com/2025/09/07/world-news/russi...,78
148,2025-09-07-03-00-00 +0000,wsj,When President Trump unveiled his EU trade dea...,https://www.wsj.com/economy/trade/the-eus-trad...,72
117,2025-09-07-09-34-43 +0000,cbc,Russia's largest air attack on Ukraine sets fi...,https://www.cbc.ca/news/world/russia-mass-assa...,61
55,2025-09-07-16-31-16 +0000,wapo,Russia’s largest-ever air attack on Ukraine bu...,https://www.washingtonpost.com/world/2025/09/0...,59


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
93,83,2025-09-07-12-19-07 +0000,nypost,South Korean workers arrested in ICE raid at G...,https://nypost.com/2025/09/07/us-news/south-ko...
117,61,2025-09-07-09-34-43 +0000,cbc,Russia's largest air attack on Ukraine sets fi...,https://www.cbc.ca/news/world/russia-mass-assa...
148,42,2025-09-07-03-00-00 +0000,wsj,When President Trump unveiled his EU trade dea...,https://www.wsj.com/economy/trade/the-eus-trad...
158,37,2025-09-07-01-00-00 +0000,wsj,"Trump has so far stood by RFK Jr., the health ...",https://www.wsj.com/politics/policy/trump-said...
127,35,2025-09-07-08-42-01 +0000,nypost,Pope Leo XIV names first two saints of his pon...,https://nypost.com/2025/09/07/world-news/pope-...
77,30,2025-09-07-14-00-00 +0000,wsj,The story of American capitalism may have begu...,https://www.wsj.com/economy/capitalism-america...
108,28,2025-09-07-10-21-00 +0000,bbc,Japan's Prime Minister Shigeru Ishiba resigns ...,https://www.bbc.com/news/articles/c98edpz2yn9o...
41,24,2025-09-07-17-09-14 +0000,nypost,"Undersea cables cut in Red Sea, interrupting i...",https://nypost.com/2025/09/07/world-news/under...
185,24,2025-09-06-20-59-15 +0000,nypost,"Israel issues evacuation orders, tells Palesti...",https://nypost.com/2025/09/06/world-news/israe...
71,24,2025-09-07-15-08-17 +0000,nypost,Russian attack on Ukraine’s government HQ in l...,https://nypost.com/2025/09/07/world-news/russi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
